# Processamento dos dados do Corpus

## Extração dos dados

Foi utilizado o brwac.vert como fonte para os dados de treinamento dos modelos. Para obter acesso a ele, siga as instruções em:

 [https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC)

In [4]:
import datasets
import pandas as pd

brwac_folder = './data'


In [4]:
!7z x brwac.vert.7z -odata


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=pt_BR.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx   (810F81),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./brwa                1 file, 4778375677 bytes (4558 MiB)

Extracting archive: ./brwac/brwac.vert.7z
--
Path = ./brwac/brwac.vert.7z
Type = 7z
Physical Size = 4778375677
Headers Size = 130
Method = LZMA2:26
Solid = -
Blocks = 1

      0% - brwac.ver                  1% - brwac.ver                  2% - brwac.ver                  3% - brwac.ver                  4% - brwac.ver                  5% - brwac.ver                  6% - brwac.ver                  7% - brwac.ver                  8% - brwac.ver                  9% - brwac.ver                 10% - brwac.ver                 11% - brwac.ver                 12% - brwac.ver                 13% - brwac.ver                 14% - brwac.ver                 15% - brwac.ver            

In [16]:
data = datasets.load_dataset("brwac", data_dir="data")

Generating train split:   0%|          | 0/3530796 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

In [20]:
print(data['train'])

Dataset({
    features: ['doc_id', 'title', 'uri', 'text'],
    num_rows: 3530796
})


In [21]:
data['train'].to_csv('brwac.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/3531 [00:00<?, ?ba/s]

## Recorte dos dados

Para facilitar a classificação dos dados, foi escolhido um subset do brwac contendo apenas dados de subpáginas UOL.

In [ ]:
classes = {
    'caras': 0,
    'esporte': 0,
    'noticias': 0,
    'folha': 0,
    'educacao': 0,
    'jogos': 0,
    'tecnologia': 0,
}

base_uri = 'uol.com.br'

### Cria os arquivos CSV das classes

In [ ]:
headers = ['id', 'title', 'uri', 'text']

df = pd.DataFrame(columns=headers)

for c in classes.keys():
    df.to_csv(f"{c}.brwac.csv", index=False)

In [ ]:
ck_size = 100000
df_iterator = pd.read_csv('brwac.csv', chunksize=ck_size)
    
for df in df_iterator:
    for c in classes.keys():
        df[df['uri'].str.contains(f'{c}.{base_uri}')].to_csv(f"{c}.brwac.csv", mode='a', header=False, index=False)
        
        classes[c] += df[df['uri'].str.contains(f'{c}.{base_uri}')].shape[0]
        print(f'{c}.{base_uri}: {classes[c]}')
        
    print()
    